In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchvision

from utils import *
from gaussian_training import *

from experiments.dataloaders import *

%load_ext autoreload
%autoreload 2

In [4]:
from ray import tune


ImportError: ray.tune in ray > 0.7.5 requires 'tabulate'. Please re-run 'pip install ray[tune]' or 'pip install ray[rllib]'.

In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator
# import tensorflow as tf

In [ ]:
import sys, os

def get_run_file(folder_name):
    folder_path = "./runs/{}".format(folder_name)
    file_name = list(os.walk(folder_path))[0][2][0]
    path_to_events_file = os.path.join(folder_path, file_name)
    return path_to_events_file

In [ ]:
from collections import defaultdict

def tb_to_pd(path):
    
    nets_dict = defaultdict(defaultdict(defaultdict(float)))

    # This example supposes that the events file contains summaries with a
    # summary value tag 'loss'.  These could have been added by calling
    # `add_summary()`, passing the output of a scalar summary op created with
    # with: `tf.scalar_summary(['loss'], loss_tensor)`.
    for e in summary_iterator(path_to_events_file):
        for v in e.summary.value:
            net_num = int(v.tag.split("_")[1]) # Todo make this /
            name = v.tag.split("/")[1]

            if e.step not in nets_dict_potential[net_num]:
                nets_dict_potential[net_num][e.step] = {}
            nets_dict_potential[net_num][e.step][name] = v.simple_value
                   
        

In [ ]:
def temp_tb_to_dict(path_to_events_file):
    nets_dict_potential= [{} for _ in range(config["num_nets"])] # TODO make this iter dicts
    nets_dict_acc = [{} for _ in range(config["num_nets"])] # TODO make this iter dicts

    for e in summary_iterator(path_to_events_file):
        for v in e.summary.value:
            if ("Potential" in v.tag):
                net_num = int(v.tag.split("_")[1]) # Todo make this /
                name = v.tag.split("/")[1]

                if e.step not in nets_dict_potential[net_num]:
                    nets_dict_potential[net_num][e.step] = {}
                nets_dict_potential[net_num][e.step][name] = v.simple_value
            elif "Accuracy" in v.tag:
                net_num = int(v.tag.split("_")[1]) # Todo make this /

                nets_dict_acc[net_num][e.step] = v.simple_value  
                
    return nets_dict_potential, nets_dict_acc
        

In [ ]:
def get_last_acc_potential(nets_dict_potential, nets_dict_acc):
    w = []
    a = []
    for nn in range(len(nets_dict_acc)):
        last_step = int(list(nets_dict_acc[nn].keys())[-2])
        a.append(nets_dict_acc[nn][last_step])
        w.append(nets_dict_potential[nn][last_step]["total"])
    return w, a

In [ ]:
def get_last(folder_name):
    file_path = get_run_file(folder_name)
    potential, acc = temp_tb_to_dict(file_path)
    return get_last_acc_potential(potential, acc)

In [ ]:
sharp_potential, sharp_acc = get_last("Mar18_01-44-57_Daniels-MacBook-Pro-4.local")
flat_potential, flat_acc = get_last("Mar18_01-54-06_Daniels-MacBook-Pro-4.local")


In [ ]:
# plt.scatter(without_potential_last, without_acc_last, label="without")
# plt.scatter(sharp_potential_last, sharp_acc_last, label="sharp")
plt.scatter(flat_potential, flat_acc, label="flat")
plt.scatter(sharp_potential, sharp_acc, label="sharp")
plt.legend()

plt.show()

In [ ]:
from scipy.stats.stats import pearsonr   
print(pearsonr(a,w))

In [ ]:
import pickle

def get_models(experiment_name, step):
    path = "./models/gaussian/{}".format(experiment_name)
    nets_dict = {}
    for root, dirs, files in os.walk(path):
        for name in files:
            name_split_underscore = name.split("_")
            if name_split_underscore[-1].split(".")[0] == str(step):
                file_path = os.path.join(root, name)
                with open(file_path, "rb") as f:
                    net = torch.load(f)
                nets_dict[name_split_underscore[1]] = net
    return nets_dict

In [ ]:
exp_name = "Mar_14_16-21-33_Daniels-MacBook-Pro-4.local"
models = get_models(exp_name, 3149)

In [ ]:
import torch
from hessian_eigenthings import compute_hessian_eigenthings

model = models["0"]
dataloader = data[0]
loss = torch.nn.CrossEntropyLoss()

num_eigenthings = 20  # compute top 20 eigenvalues/eigenvectors

eigenvals, eigenvecs = compute_hessian_eigenthings(model, dataloader,
                                                   loss, num_eigenthings,use_gpu=False)

In [ ]:
plt.plot(eigenvals)
plt.show()